### Pipeline that checks if a number is Non-Prime and Outputs the Prime Factors

##### Process Flow in Creating a YData Pipeline  -

- Define the Function to Execute inside your Pipeline block 
- Create the Pipeline Block component(s)
- Create the component task (Multiple tasks with different input Parameters can be created from the same Pipeline    component
- Create the Pipeline definition with the Execution sequence
- Compile the Pipeline 
- Upload the Generated .tar.gz file to your YData Pipelines and Run!

In [ ]:
# The kfp-python package provides the Interface to Generate and Run Pipelines

import kfp
import typing

#### Read the Number Input and Pass it to the next Pipeline Block

In [ ]:
from kfp.components import *

# This is the Function definition which will run inside your first Pipeline block
#Input Parameters are defined just like a normal Python function definition

def import_number(numb: int) -> int: #Single Output Parameters are defined as shown
    num = numb
    return num      #return the output Parameter

In [ ]:
#kfp.components subclass is used to create Pipeline components from Python definitions/functions 
#func_to_container_op packages the python function into a Pipeline Block which runs as a Kubernetes Pod

kfp_import_number = kfp.components.func_to_container_op(func = import_number, 
                                                          output_component_file = './imp_num.yaml',  
                             #this yaml component file is the Kubernetes pod definition for your Pipeline block 
                                                       packages_to_install = ['sympy==1.5.1'])
                                #kfp uses a Docker Tensorflow container image
                                #list the packages that are not inherently installed into that container
                                #image over here, so that they are installed at the time of 
                                #container creation. Be sure to also include the correct package version if it 
                                #is not the most recent one.

#### Check if Number is a Prime or a Consonant

In [ ]:
from typing import NamedTuple

#Named Tuple is used for declaring Multiple Parameter Outputs

from kfp.components import *

def is_prime(numb: int) -> NamedTuple('S_outs', [('num', int), ('result', bool)]): 
                            #Multiple Output Parameters are defined using Named Tuple
    
    #Note that it is also necessary to import the packages in addition to listing them in the
    #func_to_container_op definition
    
    import sympy
    
    num = numb
    if(sympy.isprime(num)):
        str_isprime = "{} is a Prime Number".format(num)
    else:
        str_isprime = "{} is a Consonant".format(num)
    
    print(str_isprime)   #Printed Outputs can be viewed in the Pipeline Block's 'Logs' tab
    
    return (num, sympy.isprime(num))  #Return the Multiple Parameters as a Tuple

In [ ]:
kfp_is_prime = kfp.components.func_to_container_op(func = is_prime, 
                                                          output_component_file = './is_prime.yaml',
                                                       packages_to_install = ['sympy'])


#### Conditional Block - Only executed if the Number is Prime

In [ ]:
def prime_print(numb: int) -> str:
    s = "{} is Prime".format(numb)
    print(s)
    return s

In [ ]:
kfp_prime_print = kfp.components.func_to_container_op(func = prime_print, 
                                                          output_component_file = './prime_print.yaml')


#### Consonant Number's Prime Factors are Returned

In [ ]:
from typing import NamedTuple
from kfp.components import *

def prime_fact(numb: int) -> NamedTuple('S_outs', [('num', int), ('prime_factors', list)]):
    import sympy
    num = numb
    l_prime = sympy.primefactors(num)
    print("Factors of {} are: ".format(num))
    for x in l_prime:
        print("{} ".format(x), end=" ")
    return (num, l_prime)

In [ ]:
kfp_prime_fact = kfp.components.func_to_container_op(func = prime_fact, 
                                                          output_component_file = './prime_fact.yaml',
                                                  packages_to_install = ['sympy'])


#### Exit Handler

In [ ]:
#The exit Handler function is the Pipeline-block that will execute regardless of successful or failed execution 
#of the Pipeline

def exit_handle(msg: str):
    print(msg)

In [ ]:
kfp_exit_handle = kfp.components.func_to_container_op(func = exit_handle, 
                                                      output_component_file = "./ext_hdle.yaml")

#### Defining the Pipeline Execution Sequence and Input-Output scheme

In [ ]:
#kfp.dsl is used to create the Pipeline Definition and the Sequence of Execution for the Pipeline Blocks

import kfp.dsl as dsl

#@dsl decorator can also be used

@dsl.pipeline(name='Prime Factorisation',description='Prime Factorizes if given number is a Consonant')
def Primefact_func(enter_number: int):  #The input parameter(s) defined here will be accessible to the User running
                                        #the pipeline from the YData Pipelines Dashboard
    
    exit_task = kfp_exit_handle('Operation completed')  #Exit task is defined first - this is the task 
    
    #It is important to follow the correct sequence in defining the Operational sequence of the Pipeline Blocks
    #Keeping in mind their relative Input-Output schemes
    #dsl.ExitHandler is used to create an Unconditionally Executable Block - with the 
    
    with dsl.ExitHandler(exit_task):
        imp_num_task = kfp_import_number(enter_number)
        is_prime_task = kfp_is_prime(imp_num_task.output)
        
        #dsl.Condition is used to create conditional execution of a Pipeline Block
        #If Number is Prime, only this block will execute, and Print output 'Prime'
        with dsl.Condition(is_prime_task.outputs['result']==True):
            prime_print_task = kfp_prime_print(is_prime_task.outputs['num'])
        
        #If Number is non-Prime, this block will execute, and return the Prime Factors of the Number
        with dsl.Condition(is_prime_task.outputs['result']==False):
            prime_fact_task = kfp_prime_fact(is_prime_task.outputs['num'])

#For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
#For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax

#### Compiling the Pipeline

In [ ]:
#pipeline_func should be the same as the Pipeline definition name above

pipeline_func = Primefact_func
pipeline_filename = pipeline_func.__name__+'.pipeline.tar.gz'   #Filename can be different if you want

import kfp.compiler as comp
comp.Compiler().compile(pipeline_func, pipeline_filename)

#This will generate the .tar.gz file for the Pipeline, which should be uploaded to the YData Pipelines Dashboard
#using the procedure as in the Tutorial document